<a href="https://colab.research.google.com/github/gacemiamine77/gacemiamine77/blob/main/EE_to_drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import ee

In [7]:
 ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=xUm5egd5r2YsRQthQm4hm-8KtqR83FOEH3rt3wNrgQg&tc=O0mMSkkwqXqhwEaXEzJre0fVa5Ei6WgCmeUHWagt2BI&cc=qA3-5bnVpSWFUur7VWAnSqQol6p5z2QmzIPw3B7LhbE

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhgJc4YCQ7x7R0geli5FqqoPF8L6tRnZ8ttP7ngVHMhQmgsJsJa5Pg

Successfully saved authorization token.


In [15]:
# Load a landsat image and select three bands.
landsat = ee.Image('LANDSAT/LC08/C02/T1_L1/LC08_197035_20220527')\
            .select(['B4', 'B3', 'B2'])

#LC08_L1TP_197035_20220527_20220603_02_T1
# Create a geometry representing an export region.
geometry = ee.Geometry.Rectangle([36.027, 0.03, 35.89, 0.247])
center = geometry.centroid().getInfo()['coordinates']
center.reverse()

In [17]:
L8coll = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filter(ee.Filter.lt('CLOUD_COVER',5)).filterBounds(geometry)

In [ ]:
region = ee.Geometry.BBox(36, -0.04, 35.8,0.27)

task = ee.batch.Export.image.toDrive(**{
    'image': landsat,
    'description': 'i1mageToDriveExample',
    'folder':'Example_fol',
    'scale': 100,
    'region': geometry.getInfo()['coordinates']
})
task.start()

In [20]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 25.8 MB/s 
     |████████████████████████████████| 1.3 MB 48.5 MB/s 
     |████████████████████████████████| 130 kB 55.0 MB/s 
     |████████████████████████████████| 8.8 MB 52.0 MB/s 
     |████████████████████████████████| 187 kB 61.7 MB/s 
     |████████████████████████████████| 1.2 MB 54.9 MB/s 
     |████████████████████████████████| 95 kB 3.7 MB/s 
     |████████████████████████████████| 3.3 MB 42.4 MB/s 
     |████████████████████████████████| 3.0 MB 59.4 MB/s 
     |████████████████████████████████| 98 kB 8.8 MB/s 
     |████████████████████████████████| 99 kB 11.9 MB/s 
     |████████████████████████████████| 46 kB 4.0 MB/s 
     |████████████████████████████████| 128 kB 69.6 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 344 kB 68.4 MB/s 
     |████████████████████████████████| 

In [22]:
import ee
import geemap

In [23]:
Map = geemap.Map()

In [24]:
Map = geemap.Map(center=[36,0], zoom=4)
Map

Map(center=[36, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [39]:
collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate('2013-01-01', '2022-06-30').filter(ee.Filter.eq('WRS_PATH', 198)).filter(ee.Filter.eq('WRS_ROW', 35))
print(collection.size().getInfo())

191


In [40]:
median = collection.median()

In [41]:
def apply_scale_factors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

In [42]:
dataset = apply_scale_factors(median)

In [43]:
vis_natural = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

vis_nir = {
    'bands': ['SR_B5', 'SR_B4', 'SR_B3'],
    'min': 0.0,
    'max': 0.3,
}

In [44]:
Map.addLayer(dataset, vis_natural, 'True color (432)')
Map.addLayer(dataset, vis_nir, 'Color infrared (543)')
Map

Map(bottom=6819.0, center=[34.470335121217495, 0.406494140625], controls=(WidgetControl(options=['position', '…

In [31]:
vis_params = [
    {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3},
    {'bands': ['SR_B5', 'SR_B4', 'SR_B3'], 'min': 0, 'max': 0.3},
    {'bands': ['SR_B7', 'SR_B6', 'SR_B4'], 'min': 0, 'max': 0.3},
    {'bands': ['SR_B6', 'SR_B5', 'SR_B2'], 'min': 0, 'max': 0.3},
]

In [32]:
labels = [
    'Natural Color (4, 3, 2)',
    'Color Infrared (5, 4, 3)',
    'Short-Wave Infrared (7, 6 4)',
    'Agriculture (6, 5, 2)',
]